In [1]:
# Script to post-process high-resolution WRF model output.

# Major tasks include computing the following for selected variables:
#   1. domain-averages to produce time series
#   2. vertical integrals
#   3. pressure-level vertical interpolation

In [2]:
from netCDF4 import Dataset
import numpy as np
from wrf import getvar, vinterp, ALL_TIMES
import xarray as xr
from read_wrf_piccolo import *
from post_proc_functions import *
import os

In [3]:
do_2d_special = True
do_3d_vars = False
do_3d_special = False

In [4]:
#### Directories and model output specs

case = "sept1-4"
test_process = "ctl"

wrfdir = "/glade/campaign/univ/uokl0053/"+test_process+"/"
# wrfdir = "/ourdisk/hpc/radclouds/auto_archive_notyet/tape_2copies/wrf-ideal/largedom2/"+test_process+"/"
outdir = wrfdir+"post_proc/"
os.makedirs(outdir, exist_ok=True)

# # Get WRF file list, dimensions
wrftag = "wrfout_d01"
wrffiles = get_wrf_file_list(wrfdir, wrftag)
lat, lon, nx1, nx2, nz, npd = wrf_dims(wrffiles[0])
nfiles = len(wrffiles)

# New vertical dimension for pressure levels
dp = 25 # hPa
pres = np.arange(1000, 25, -dp)
nznew = len(pres)

# Get variable lists
vars3d = var_list_3d()

In [5]:
if do_2d_special:

    # Read in variable from WRF files
    # for ifile in range(nfiles):
    for ifile in range(1):

        # Open the WRF file
        file = wrffiles[ifile]
        print("Processing "+file)
        ds = Dataset(file)

        qv = getvar(ds, "QVAPOR", timeidx=ALL_TIMES)#, cache=cache)
        pwrf = getvar(ds, "p", units='Pa', timeidx=ALL_TIMES)#, cache=cache)
        # hght = getvar(dset, "zstag", units='m', timeidx=ALL_TIMES)#, cache=cache)
        # tmpk = getvar(dset, "tk", timeidx=ALL_TIMES)#, cache=cache)
        # rho = density_moist(tmpk, qv, pwrf)

        # Get dz
        # dz = np.zeros(qv.shape)
        # for iz in range(nz):
        #     dz[:,iz] = hght[:,iz+1] - hght[:,iz]
        # Get dp
        dp = pwrf.differentiate('bottom_top')*-1

        # Read in variables

        # rainrate
        var = getvar(ds, 'RAINNC', timeidx=ALL_TIMES)
        if ifile == 0:
            rainnc_all = var
        else:
            rainnc_all = xr.concat((rainnc_all, var), 'Time')
        # pclass
        var = wrf_pclass(ds, dp)
        if ifile == 0:
            pclass_all = var
        else:
            pclass_all = xr.concat((pclass_all, var), 'Time')
        # pw
        var = vert_int(qv, dp)
        if ifile == 0:
            pw_all = var
        else:
            pw_all = xr.concat((pw_all, var), 'Time')
        # pw_sat
        qvsat = get_rv_sat(ds, pwrf)
        qvsat = xr.DataArray(qvsat, coords=qv.coords, dims=qv.dims, attrs=qv.attrs)
        var = vert_int(qvsat, dp)
        if ifile == 0:
            pw_sat_all = var
        else:
            pw_sat_all = xr.concat((pw_sat_all, var), 'Time')

        ds.close()

    # Calculate rain rate as centered difference
    nt_all = rainnc_all.shape[0]
    rainrate = rainnc_all.copy()
    rainrate[0] = 0
    rainrate[nt_all-1] = np.nan
    rainrate[1:-1] = (rainnc_all.values[2:] - rainnc_all.values[:-2])*0.5
    rainrate *= npd # mm/time step --> mm/day

    # Write out the variables
    var_name='pclass'
    write_ncfile(outdir, pclass_all, var_name)
    var_name='rainrate'
    write_ncfile(outdir, rainrate, var_name)
    var_name='pw'
    write_ncfile(outdir, pw_all, var_name)
    var_name='pw_sat'
    write_ncfile(outdir, pw_sat_all, var_name)

Processing /ourdisk/hpc/radclouds/auto_archive_notyet/tape_2copies/wrf-ideal/largedom2/ctl/wrfout_d01_2007-06-01_00:00:00


In [8]:
if do_3d_vars:

    for ivar in vars3d[0:1]:

        varname_str = ivar.upper()

        # Read in variable from WRF files
        # for ifile in range(nfiles):
        print(ivar.upper())
        for ifile in range(1):
            wrfdset = Dataset(wrffiles[ifile])
            var = wrfdset.variables[ivar.upper()][...]
            wrfdset.close()
            if ifile == 0:
                var_all = var
            else:
                var_all = np.concatenate((var_all, var), axis=0)

In [26]:
dset = Dataset(wrffiles[0])
qv = getvar(dset, "QVAPOR", timeidx=ALL_TIMES)#, cache=cache)
qv_interp = vinterp(dset, qv, 'p', pres, extrapolate=True, timeidx=ALL_TIMES)#, cache=cache)
dset.close()
qv_interp.shape

(48, 39, 266, 266)

In [11]:
print('Time without cache: 1:23')
print('Time with cache: 2:23')

Time without cache: 1:32
Time with cache: 2:23


In [31]:
outfile = outdir+"qv_interp.nc"
qv_interp.to_netcdf(outfile, mode='w', format='NETCDF4')


TypeError: Invalid value for attr 'projection': RotatedLatLon(stand_lon=0.0, moad_cen_lat=0.0, truelat1=0.0, truelat2=0.0, pole_lat=0.0, pole_lon=0.0). For serialization to netCDF files, its value must be of one of the following types: str, Number, ndarray, number, list, tuple